In [2]:
!pip install openpyxl   # for .xlsx
!pip install xlrd       # for .xls



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [39]:
import pandas as pd
import seaborn as sns  #load dataset
import numpy as np #statistical calculations
import matplotlib.pyplot as plt # for visualization
import psycopg2
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
from time import sleep

In [4]:
# loading the original dataset

data = pd.read_excel("baederleben_berlin.xlsx")
data.head()

,Bad-ID,Badtyp,eu_badegewaesser,Name,Straße,Postleitzahl,Ort,Ansprechpartner,Telefon,E-Mail-Adresse,...,Wettkampftauglichkeit Schwimmen,Wettkampftauglichkeit Wasserball,Wettkampftauglichkeit Springen,Wettkampftauglichkeit Rettungsschwimmen,Wettkampftauglichkeit Synchronschwimmen,Wettkampftauglichkeit Tauchen,Badpate vorhanden,Erweiterungen,Sanierungen,Förderungen
0,472,Naturbad,NaN,Strandbad Lübars,Am Freibad 9,13469,Berlin,NaN,030 53086035,info@strandbad-luebars.de,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,NaN,NaN,NaN
1,473,Hallenbad,nein,Kleine Schwimmhalle Wuhlheide,An der Wuhlheide 161,12459,Berlin,NaN,030 22190011,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,NaN,NaN,NaN
2,474,Kombibad,nein,Kombibad Mariendorf,Ankogelweg 95,12107,Berlin,Herr Ramm,030 74067830,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,NaN,NaN,"Förderung SJK 2022, 6.000.000 €"
3,475,Hallenbad,nein,Schwimmhalle Anton-Saefkow-Platz,Anton-Saefkow-Platz 1,10369,Berlin,Herr Heinz,030 9729169,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,NaN,NaN,NaN
4,476,Hallenbad,nein,Stadtbad Kreuzberg - Baerwaldbad,Baerwaldstraße 64-67,10961,Berlin,Herr Uffelmann,030 69564549,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nein,NaN,NaN,NaN


## Data Exploration


In [5]:
data.shape

(144, 136)

In [6]:
data.describe()

,Bad-ID,Postleitzahl,Gemeindekennzahl,Längengrad,Breitengrad,Öffnungsstunden pro Jahr,Baujahr,Baukosten bei Fertigstellung,Baukosten heute (Euro),Besucher,...,Anteil VHS,Anteil Kommerzielle Nutzung,Anteil Sonstige Nutzer,Anzahl Startblöcke,Wettkampftauglichkeit Schwimmen,Wettkampftauglichkeit Wasserball,Wettkampftauglichkeit Springen,Wettkampftauglichkeit Rettungsschwimmen,Wettkampftauglichkeit Synchronschwimmen,Wettkampftauglichkeit Tauchen
count,144.000000,144.000000,144.0,144.000000,144.000000,0.0,49.000000,1.0,0.0,2.000000,...,0.0,0.0,0.0,38.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,5096.506944,12565.152778,11000000.0,13.364897,52.494014,NaN,1960.877551,6800000.0,NaN,542000.000000,...,NaN,NaN,NaN,6.473684,NaN,NaN,NaN,NaN,NaN,NaN
std,4010.713202,1276.171872,0.0,0.138273,0.055093,NaN,25.381779,NaN,NaN,647709.811567,...,NaN,NaN,NaN,4.221802,NaN,NaN,NaN,NaN,NaN,NaN
min,472.000000,10115.000000,11000000.0,13.114743,52.373220,NaN,1896.000000,6800000.0,NaN,84000.000000,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1043.750000,12105.000000,11000000.0,13.255845,52.445267,NaN,1950.000000,6800000.0,NaN,313000.000000,...,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,6987.500000,12573.000000,11000000.0,13.350150,52.492320,NaN,1970.000000,6800000.0,NaN,542000.000000,...,NaN,NaN,NaN,5.500000,NaN,NaN,NaN,NaN,NaN,NaN
75%,8500.250000,13587.000000,11000000.0,13.452025,52.539145,NaN,1980.000000,6800000.0,NaN,771000.000000,...,NaN,NaN,NaN,7.750000,NaN,NaN,NaN,NaN,NaN,NaN
max,10510.000000,14199.000000,11000000.0,13.735059,52.628680,NaN,2000.000000,6800000.0,NaN,1000000.000000,...,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Columns: 136 entries, Bad-ID to Förderungen
dtypes: float64(41), int64(3), object(92)
memory usage: 153.1+ KB


In [8]:
print(data.dtypes)

Bad-ID                             int64
Badtyp                            object
eu_badegewaesser                  object
Name                              object
Straße                            object
                                  ...   
Wettkampftauglichkeit Tauchen    float64
Badpate vorhanden                 object
Erweiterungen                     object
Sanierungen                       object
Förderungen                       object
Length: 136, dtype: object


## Data Processing

In [9]:

# First, to consider these columns  based on their importance for mapping and analysis

important_columns = [
    "Bad-ID",          # Unique ID for each pool/facility
    "Name",            # Facility name
    "Badtyp",          # Type (indoor, outdoor, open water, etc.)
    "Straße",          # Street address
    "Postleitzahl",             # Postal code
    "Ort",        # Local neighborhood / quarter
    "Bezirk",          # Borough / district
    "Breitengrad",        # Latitude for map placement
    "Längengrad",       # Longitude for map placement
    "eu_badegewaesser",# EU-designated bathing site flag
    "Barrierefreiheit",# Accessibility info
    "Wasserqualitaet", # Water quality rating
    "Öffnungsstunden pro Jahr",  # Opening hours (if present in dataset)
    "Baujahr",          # Year built (for historical context)
    "Ganzjährig geöffnet",
    "Name des Eigentümers",
    "Ermäßigung Kind",  # Discount for children
    "Ermäßigung Familie",  # family discount
    "Ermäßigung Behinderte",  # discount for behinder
    
]

In [10]:
# To Keep only these columns from the  dataset for further assessment
columns_to_use = [col for col in important_columns if col in data.columns]

filtered_data = data[columns_to_use]

filtered_data.head()

,Bad-ID,Name,Badtyp,Straße,Postleitzahl,Ort,Breitengrad,Längengrad,eu_badegewaesser,Öffnungsstunden pro Jahr,Baujahr,Ganzjährig geöffnet,Name des Eigentümers,Ermäßigung Kind,Ermäßigung Familie,Ermäßigung Behinderte
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,Berlin,52.61824,13.33519,NaN,NaN,NaN,nein,Henry Arzig,ja,ja,ja
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,Berlin,52.45993,13.53965,nein,NaN,1990.0,ja,NaN,ja,nein,ja
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,Berlin,52.41972,13.40154,nein,NaN,1974.0,ja,NaN,ja,ja,ja
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,Berlin,52.53093,13.47184,nein,NaN,1980.0,ja,NaN,ja,nein,NaN
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,Berlin,52.49451,13.40432,nein,NaN,1955.0,ja,Förderverein,ja,ja,ja


In [27]:
filtered_data.shape

(144, 16)

In [28]:
# converting columns to lower case and improve them into standards
filtered_data.columns = filtered_data.columns.str.lower().str.replace(' ', '_')


In [29]:
filtered_data.head()

,pool_id,name,pool_type,street,postal_code,city,latitude,longitude,eu_bathing_water,annual_opening_hours,year_built,open_all_year,owner_name,discount_children,discount_family,discount_disabled
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,Berlin,52.61824,13.33519,NaN,NaN,NaN,False,Henry Arzig,ja,ja,ja
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,Berlin,52.45993,13.53965,nein,NaN,1990.0,True,NaN,ja,nein,ja
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,Berlin,52.41972,13.40154,nein,NaN,1974.0,True,NaN,ja,ja,ja
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,Berlin,52.53093,13.47184,nein,NaN,1980.0,True,NaN,ja,nein,NaN
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,Berlin,52.49451,13.40432,nein,NaN,1955.0,True,Förderverein,ja,ja,ja


In [30]:
# Converting the columns name from German to English
rename_map = {
    'bad-id': 'pool_id',
    'name': 'name',
    'badtyp': 'pool_type',
    'straße': 'street',
    'postleitzahl': 'postal_code',
    'ort': 'city',
    'breitengrad': 'latitude',
    'längengrad': 'longitude',
    'eu_badegewaesser': 'eu_bathing_water',
    'öffnungsstunden_pro_jahr': 'annual_opening_hours',
    'baujahr': 'year_built',
    'ganzjährig_geöffnet': 'open_all_year',
    'name_des_eigentümers': 'owner_name',
    'ermäßigung_kind': 'discount_children',
    'ermäßigung_familie': 'discount_family',
    'ermäßigung_behinderte': 'discount_disabled'
}

In [15]:
filtered_data.rename(columns=rename_map, inplace=True)

print(filtered_data.columns.tolist())



['pool_id', 'name', 'pool_type', 'street', 'postal_code', 'city', 'latitude', 'longitude', 'eu_bathing_water', 'annual_opening_hours', 'year_built', 'open_all_year', 'owner_name', 'discount_children', 'discount_family', 'discount_disabled']


/var/folders/z5/4t7v7msd5dq10zw9x1f85jx80000gn/T/ipykernel_24302/659988775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.rename(columns=rename_map, inplace=True)


In [16]:
filtered_data.head()

,pool_id,name,pool_type,street,postal_code,city,latitude,longitude,eu_bathing_water,annual_opening_hours,year_built,open_all_year,owner_name,discount_children,discount_family,discount_disabled
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,Berlin,52.61824,13.33519,NaN,NaN,NaN,nein,Henry Arzig,ja,ja,ja
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,Berlin,52.45993,13.53965,nein,NaN,1990.0,ja,NaN,ja,nein,ja
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,Berlin,52.41972,13.40154,nein,NaN,1974.0,ja,NaN,ja,ja,ja
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,Berlin,52.53093,13.47184,nein,NaN,1980.0,ja,NaN,ja,nein,NaN
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,Berlin,52.49451,13.40432,nein,NaN,1955.0,ja,Förderverein,ja,ja,ja


In [17]:
filtered_data.duplicated(subset='pool_id').sum()  # checking for duplicate

np.int64(0)

In [18]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pool_id               144 non-null    int64  
 1   name                  144 non-null    object 
 2   pool_type             144 non-null    object 
 3   street                141 non-null    object 
 4   postal_code           144 non-null    int64  
 5   city                  144 non-null    object 
 6   latitude              144 non-null    float64
 7   longitude             144 non-null    float64
 8   eu_bathing_water      111 non-null    object 
 9   annual_opening_hours  0 non-null      float64
 10  year_built            49 non-null     float64
 11  open_all_year         114 non-null    object 
 12  owner_name            63 non-null     object 
 13  discount_children     111 non-null    object 
 14  discount_family       108 non-null    object 
 15  discount_disabled     1

In [31]:
#filtered_data['postal_code'] = filtered_data['postal_code'].astype(str)
filtered_data.loc[:, 'postal_code'] = filtered_data['postal_code'].astype(str)




In [32]:
filtered_data.loc[:, 'city'] = filtered_data['city'].str.title()


In [33]:
# Convert yes/no or German terms to Boolean
yes_values = ['Ja', 'Yes', 'yes', 'ja', True]
filtered_data.loc[:, 'open_all_year'] = filtered_data['open_all_year'].apply(lambda x: True if x in yes_values else False)


In [34]:
filtered_data.head()

,pool_id,name,pool_type,street,postal_code,city,latitude,longitude,eu_bathing_water,annual_opening_hours,year_built,open_all_year,owner_name,discount_children,discount_family,discount_disabled
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,Berlin,52.61824,13.33519,NaN,NaN,NaN,False,Henry Arzig,ja,ja,ja
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,Berlin,52.45993,13.53965,nein,NaN,1990.0,True,NaN,ja,nein,ja
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,Berlin,52.41972,13.40154,nein,NaN,1974.0,True,NaN,ja,ja,ja
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,Berlin,52.53093,13.47184,nein,NaN,1980.0,True,NaN,ja,nein,NaN
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,Berlin,52.49451,13.40432,nein,NaN,1955.0,True,Förderverein,ja,ja,ja


In [35]:
# Based on the available data, these columns were selected to ensure the dataset 
# is accurate, informative, and useful for maps, clustering, and accessibility analysis:

mapping_cols = [
    'pool_id', 'name', 'pool_type', 'street', 'postal_code', 'city',
    'latitude', 'longitude', 'open_all_year'
]
final_columns = filtered_data[mapping_cols]

In [36]:
final_columns.head()

,pool_id,name,pool_type,street,postal_code,city,latitude,longitude,open_all_year
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,Berlin,52.61824,13.33519,False
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,Berlin,52.45993,13.53965,True
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,Berlin,52.41972,13.40154,True
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,Berlin,52.53093,13.47184,True
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,Berlin,52.49451,13.40432,True


In [37]:
# To keep only valid geographic coordinates

final_columns = final_columns[(final_columns['latitude'].between(-90, 90)) & (final_columns['longitude'].between(-180, 180))]


In [38]:
final_columns.to_csv("pools_data_cleaned.csv", index=False)

In [40]:
final_columns.head()

,pool_id,name,pool_type,street,postal_code,city,latitude,longitude,open_all_year
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,Berlin,52.61824,13.33519,False
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,Berlin,52.45993,13.53965,True
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,Berlin,52.41972,13.40154,True
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,Berlin,52.53093,13.47184,True
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,Berlin,52.49451,13.40432,True


## Geocoding scripts

In [41]:

# Initialize Nominatim geocoder
geolocator = Nominatim(user_agent="berlin_bezirk_locator")

# Function to get the district from lat/lon
def get_bezirk(lat, lon):
    try:
        location = geolocator.reverse(
            (lat, lon),
            exactly_one=True,
            language='de',
            timeout=10
        )
        sleep(1)  # Stay within API limits (1 request/sec)
        if location and location.raw.get("address"):
            address = location.raw["address"]
            return (
                address.get("city_district") or
                address.get("borough") or
                address.get("county") or
                None
            )
        return None
    except Exception as e:
        print(f"Error for coords ({lat}, {lon}): {e}")
        return None

# Applying geocoding to each row to get the district name
final_columns["district"] = final_columns.apply(
    lambda row: get_bezirk(row["latitude"], row["longitude"])
    if pd.notnull(row["latitude"]) and pd.notnull(row["longitude"]) else None,
    axis=1
)


## Generating district ids

In [43]:

# District mapping (official codes as strings)
district_mapping = {
    'Mitte': '01',
    'Friedrichshain-Kreuzberg': '02',
    'Pankow': '03',
    'Charlottenburg-Wilmersdorf': '04',
    'Spandau': '05',
    'Steglitz-Zehlendorf': '06',
    'Tempelhof-Schöneberg': '07',
    'Neukölln': '08',
    'Treptow-Köpenick': '09',
    'Marzahn-Hellersdorf': '10',
    'Lichtenberg': '11',
    'Reinickendorf': '12'
}

# Apply mapping to create district_id column (string)
final_columns['district_id'] = final_columns['district'].map(district_mapping).astype(str)

# (Optional) Check if some districts were not mapped
#unmapped = df[~df['district'].isin(district_mapping.keys())]['district'].unique()
#if len(unmapped) > 0:
    #print("⚠️ Unmapped districts found:", unmapped)

In [46]:
final_columns.head(15)

,pool_id,name,pool_type,street,postal_code,latitude,longitude,open_all_year,district,district_id
0,472,Strandbad Lübars,Naturbad,Am Freibad 9,13469,52.61824,13.335190,False,Reinickendorf,12
1,473,Kleine Schwimmhalle Wuhlheide,Hallenbad,An der Wuhlheide 161,12459,52.45993,13.539650,True,Treptow-Köpenick,09
2,474,Kombibad Mariendorf,Kombibad,Ankogelweg 95,12107,52.41972,13.401540,True,Tempelhof-Schöneberg,07
3,475,Schwimmhalle Anton-Saefkow-Platz,Hallenbad,Anton-Saefkow-Platz 1,10369,52.53093,13.471840,True,Lichtenberg,11
4,476,Stadtbad Kreuzberg - Baerwaldbad,Hallenbad,Baerwaldstraße 64-67,10961,52.49451,13.404320,True,Friedrichshain-Kreuzberg,02
5,477,Strandbad am Weißen See,Naturbad,Berliner Allee 155,13086,52.55396,13.465830,False,Pankow,03
6,478,Sommerbad Staaken-West,Freibad,Brunsbüttler Damm 443,13591,52.53386,13.131230,False,Spandau,05
7,479,Schwimmhalle Kaulsdorf,Schulbad,Clara-Zetkin-Weg 13,12619,52.52080,13.585410,True,Marzahn-Hellersdorf,10
8,480,Sommerbad Neukölln,Freibad,Columbiadamm 160-180,10965,52.48025,13.415950,False,Neukölln,08
9,481,Schwimmhalle Finckensteinallee,Hallenbad,Finckensteinallee 73,12205,52.43225,13.297910,False,Steglitz-Zehlendorf,06


In [45]:
final_columns.drop(columns=["city"], inplace=True)

In [51]:
final_columns[ 'pool_id'] = final_columns['pool_id'].astype(str)


In [53]:
final_columns.to_csv("berlin_pools_final_dataset.csv", index=False)

## Designing Data Schema

In [54]:
engine = create_engine(
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

final_columns.to_sql(
    name='berlin_pools',
    con=engine,
    schema='test_berlin_data',
    if_exists='replace',
    index=False
)
print("pools data uploaded successfully!")


pools data uploaded successfully!


In [55]:
check_df = pd.read_sql("SELECT * FROM test_berlin_data.berlin_pools LIMIT 5;", engine)
print(check_df)


  pool_id                              name  pool_type                 street  \
0     472                  Strandbad Lübars   Naturbad           Am Freibad 9   
1     473     Kleine Schwimmhalle Wuhlheide  Hallenbad   An der Wuhlheide 161   
2     474               Kombibad Mariendorf   Kombibad          Ankogelweg 95   
3     475  Schwimmhalle Anton-Saefkow-Platz  Hallenbad  Anton-Saefkow-Platz 1   
4     476  Stadtbad Kreuzberg - Baerwaldbad  Hallenbad   Baerwaldstraße 64-67   

  postal_code  latitude  longitude  open_all_year                  district  \
0       13469  52.61824   13.33519          False             Reinickendorf   
1       12459  52.45993   13.53965           True          Treptow-Köpenick   
2       12107  52.41972   13.40154           True      Tempelhof-Schöneberg   
3       10369  52.53093   13.47184           True               Lichtenberg   
4       10961  52.49451   13.40432           True  Friedrichshain-Kreuzberg   

  district_id  
0          12  
1     

In [57]:
final_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pool_id        144 non-null    object 
 1   name           144 non-null    object 
 2   pool_type      144 non-null    object 
 3   street         141 non-null    object 
 4   postal_code    144 non-null    object 
 5   latitude       144 non-null    float64
 6   longitude      144 non-null    float64
 7   open_all_year  144 non-null    object 
 8   district       144 non-null    object 
 9   district_id    144 non-null    object 
dtypes: float64(2), object(8)
memory usage: 11.4+ KB
